In [1]:
import psycopg2
from config import config

In [2]:
def connect():
    '''Connect to the PostGresQL Database Server'''
    conn = None
    try:
        params = config()
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        db_version = cur.fetchone()
        print(db_version)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

if __name__=='__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.5.25 on x86_64-apple-darwin21.1.0, compiled by Apple clang version 13.0.0 (clang-1300.0.29.3), 64-bit',)
Database connection closed.


In [6]:
def select_all():
    '''Querying the PostGreSQL Database'''
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute('''SELECT * FROM actor LIMIT 20;''')
        rows = cur.fetchall()
        print(rows)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

if __name__=='__main__':
    select_all()

[(1, 'Penelope', 'Guiness', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (2, 'Nick', 'Wahlberg', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (3, 'Ed', 'Chase', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (4, 'Jennifer', 'Davis', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (5, 'Johnny', 'Lollobrigida', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (6, 'Bette', 'Nicholson', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (7, 'Grace', 'Mostel', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (8, 'Matthew', 'Johansson', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (9, 'Joe', 'Swank', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (10, 'Christian', 'Gable', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (11, 'Zero', 'Cage', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (12, 'Karl', 'Berry', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000)), (13, 'Uma', 'Wood', datetime.datetime(2013, 5, 26, 14, 47, 57, 620000))